In [1]:
5+5

10

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [5]:
extracted_data=load_pdf("data/")

In [6]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)

In [8]:
def download_embedding_model():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embedding=download_embedding_model()

c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\akhil.goud.bathini\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WA

In [10]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [11]:
from langchain import embeddings
persist_directory='db'
vectordb=Chroma.from_documents(documents=text_chunks,embedding=embedding,persist_directory=persist_directory)

In [12]:
vectordb=None

In [13]:
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [14]:
retriever=vectordb.as_retriever()

In [19]:
docs=retriever.get_relevant_documents("what is Acetaminophen")

In [21]:
docs

[Document(page_content='and reduce fever .\nPurpose\nAcetaminophen is used to relieve many kinds of\nminor aches and pains—headaches, muscle aches, back-aches, toothaches, menstrual cramps, arthritis, and theaches and pains that often accompany colds.\nGALE ENCYCLOPEDIA OF MEDICINE 2 18AcetaminophenGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 18', metadata={'page': 31, 'source': 'data\\Medical_book.pdf'}),
 Document(page_content='ORGANIZATIONS\nNational Clearinghouse on Child Abuse and Neglect Informa-\ntion. 330 C Street SW, Washington, DC 20447. (800) 392-3366.\nOTHER\nElder Abuse Prevention. <http://www.oaktrees.org/elder>.National Institute on Drug Abuse. <http://www.nida.nih.gov>.\nLaith Farid Gulli, M.D.\nBilal Nasser, M.Sc.\nAcceleration-deceleration cervical injury\nseeWhiplash\nACE inhibitors seeAngiotensin-converting\nenzyme inhibitors\nAcetaminophen\nDefinition\nAcetaminophen is a medicine used to relieve pain', metadata={'page': 31, 'source': 'data\\Medical_book.pdf'}),
 